In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

## Imports

In [11]:
import pyspark
from ifood_case.feature_engineering import FeatureEngineering
from ifood_case.model_trainer import ModelTrainer
import pyspark.sql.functions as F

ModuleNotFoundError: No module named 'optuna'

## Initialize Spark

In [4]:
spark = pyspark.sql.SparkSession.builder.appName("ifood-case").getOrCreate()

25/09/24 11:33:58 WARN Utils: Your hostname, solid resolves to a loopback address: 127.0.0.2; using 192.168.0.39 instead (on interface enp5s0)
25/09/24 11:33:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/24 11:33:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load Data

In [5]:
offers = spark.read.parquet("../data/processed/offers_processed")
transactions = spark.read.parquet("../data/processed/transactions_processed")
profile = spark.read.parquet("../data/processed/profile_processed")

## Modelling

### Feature Engieneering

In [6]:
fe = FeatureEngineering(offers, transactions, profile)

In [7]:
df, numerical_columns, categorical_columns = fe.transform()

Column account_id is string or is in exclude list: ['account_id', 'offer_id', 'time_received']
Column offer_id is string or is in exclude list: ['account_id', 'offer_id', 'time_received']
Column time_received is double or is in exclude list: ['account_id', 'offer_id', 'time_received']


In [8]:
df.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- offer_id: string (nullable = true)
 |-- time_received: double (nullable = true)
 |-- target: integer (nullable = false)
 |-- total_spend_before: double (nullable = true)
 |-- transaction_count_before: long (nullable = true)
 |-- avg_ticket_before: double (nullable = true)
 |-- max_ticket_before: double (nullable = true)
 |-- min_ticket_before: double (nullable = true)
 |-- offers_viewed_count_before: long (nullable = true)
 |-- offers_completed_count_before: long (nullable = true)
 |-- customer_conversion_rate_before: double (nullable = true)
 |-- last_offer_viewed_type: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: long (nullable = true)
 |-- credit_card_limit: double (nullable = true)
 |-- month_sin: double (nullable = true)
 |-- month_cos: double (nullable = true)
 |-- dayofweek_sin: double (nullable = true)
 |-- dayofweek_cos: double (nullable = true)
 |-- discount_value: long (nullable = true)
 |-- dura

In [9]:
numerical_columns

['target',
 'total_spend_before',
 'transaction_count_before',
 'avg_ticket_before',
 'max_ticket_before',
 'min_ticket_before',
 'offers_viewed_count_before',
 'offers_completed_count_before',
 'customer_conversion_rate_before',
 'age',
 'credit_card_limit',
 'month_sin',
 'month_cos',
 'dayofweek_sin',
 'dayofweek_cos',
 'discount_value',
 'duration',
 'min_value',
 'email',
 'social',
 'mobile',
 'web']

In [10]:
categorical_columns

['last_offer_viewed_type', 'gender', 'offer_type']

### Model Train